In [205]:
import os
import pandas as pd

cwd = os.getcwd()

# Define paths
negative_raw_data = pd.read_csv(cwd+'/testing.csv', error_bad_lines=False) 
positive_raw_data = pd.read_csv(cwd+'/tweet_info/turkey_052020_tweets_csv_hashed_2020_01.csv', nrows=200 ,low_memory=False)

b'Skipping line 105: expected 30 fields, saw 31\n'


In [206]:
positive_raw_data["label"] = 1.0
negative_raw_data["label"] = 0.0

In [207]:
#positive_raw_data.columns
#negative_raw_data.columns

In [208]:
positive_raw_data.shape

(200, 31)

In [209]:
negative_raw_data.shape

(185, 31)

In [210]:
merged_data = pd.concat([positive_raw_data, negative_raw_data], ignore_index=True)
merged_data.sample(frac=1)

,tweetid,userid,user_display_name,user_screen_name,user_reported_location,user_profile_description,user_profile_url,follower_count,following_count,account_creation_date,account_language,tweet_language,tweet_text,tweet_time,tweet_client_name,in_reply_to_userid,in_reply_to_tweetid,quoted_tweet_tweetid,is_retweet,retweet_userid,retweet_tweetid,latitude,longitude,quote_count,reply_count,like_count,retweet_count,hashtags,urls,user_mentions,label
249,1463252215804706816,927446297145159680,AK PARTİ PERTEK 🇹🇷🇹🇷,AkPertek,"Pertek, Tunceli",AK PARTİ PERTEK İLÇE BAŞKANLIĞI,remove,924,1042,2017-11-06 08:03:23+00:00,None,tr,RT @corasalih: TBMM Plan ve Bütçe Komisyonu’mu...,2021-11-23 21:04:48+00:00,Twitter for Android,remove,remove,remove,true,remove,remove,remove,remove,remove,remove,0,8,NaN,NaN,792534068,0.0
242,1463274265512448001,927446297145159680,AK PARTİ PERTEK 🇹🇷🇹🇷,AkPertek,"Pertek, Tunceli",AK PARTİ PERTEK İLÇE BAŞKANLIĞI,remove,924,1042,2017-11-06 08:03:23+00:00,None,tr,RT @AhmetSCeylan: Suriye Barış Pınarı bölgesin...,2021-11-23 22:32:25+00:00,Twitter for Android,remove,remove,remove,true,remove,remove,remove,remove,remove,remove,0,11,NaN,NaN,2234842806,0.0
359,1463277858655285257,252192548,GülseminVelidedeoğlu,Velidedeolu,ınstagram/velidedeoglusanat,"İslam, öğretmen, ebru ve katı' sanatçısı Çorum...",remove,98,109,2011-02-14 17:50:39+00:00,None,tr,RT @HaticeSudeKesk2: Korkma! Çekinme! Hüzünlen...,2021-11-23 22:46:42+00:00,Twitter for iPhone,remove,remove,remove,true,remove,remove,remove,remove,remove,remove,0,136,ErdoğanınYanındayız,NaN,1035472093683621888,0.0
96,1220447478929154054,A9BbYps2vhPzGwBdPSeuNraZfSw405ftf+OC9vfn+xY=,A9BbYps2vhPzGwBdPSeuNraZfSw405ftf+OC9vfn+xY=,A9BbYps2vhPzGwBdPSeuNraZfSw405ftf+OC9vfn+xY=,Africa,Hiç !,NaN,3065,2931,2018-02-23,tr,tr,O arabanın tavanı alçak değil mi yav\nB*ynuzla...,2020-01-23 20:45,Twitter Web App,NaN,NaN,NaN,False,NaN,NaN,absent,absent,2,12,74,19,['CHPZihniyeti'],[],[],1.0
152,1216454471624155138,30623557,mert lannister,mertlanister,NaN,şimdilerde mütevazi bir yaşam sürmekte olan es...,NaN,55678,366,2009-04-12,tr,tr,Bu şey değil mi bilgi’nin shuttle? https://t.c...,2020-01-12 20:18,Twitter for iPhone,NaN,NaN,NaN,False,NaN,NaN,absent,absent,0,0,8,0,[],[],[],1.0
221,1463274136449474568,1459514385324457987,Gözde 🎑,Gzde18181812,NaN,NaN,remove,29,28,2021-11-13 13:32:01+00:00,None,tr,RT @KantekinBerire: Gecmişin #bornova anılarıy...,2021-11-23 22:31:55+00:00,Twitter for iPad,remove,remove,remove,true,remove,remove,remove,remove,remove,remove,0,132,bornova,NaN,1432273590733492224,0.0
382,1463269723920125957,757441673001566209,Zirvadanzirveye,zirvedenzirvaya,United States,NaN,remove,799,944,2016-07-25 05:05:42+00:00,None,tr,RT @Kul_olmak_zor: Ya Rabbi bu günün tüm bela ...,2021-11-23 22:14:23+00:00,Twitter for iPhone,remove,remove,remove,true,remove,remove,remove,remove,remove,remove,0,5,NaN,NaN,2989325362,0.0
260,1463274226857689092,242957277,Ebru Sevilgen♎♑♒,EbruSevilgen,"İstanbul, Türkiye","Kurân’da, hayvandan aşağı olarak tanımlanan in...",remove,3794,337,2011-01-26 00:10:42+00:00,None,tr,Ya insansı?!🤔 https://t.co/vVwY04kGC4,2021-11-23 22:32:16+00:00,Twitter for Android,remove,remove,remove,false,remove,remove,remove,remove,remove,remove,0,0,NaN,https://t.co/vVwY04kGC4,NaN,0.0
195,1216797331422633986,GZx+ovoBhQr1geozCHIc2M+L2pXdnmXJzkwvce9Zmes=,GZx+ovoBhQr1geozCHIc2M+L2pXdnmXJzkwvce9Zmes=,GZx+ovoBhQr1geozCHIc2M+L2pXdnmXJzkwvce9Zmes=,NaN,Recep durak,NaN,309,793,2019-10-21,en,tr,"ESNAFIN ÖZGÜRLÜĞÜNÜ VERİN, \nSİCİLİNİ TEMİZLEY...",2020-01-13 19:01,Twitter for iPhone,NaN,NaN,NaN,False,NaN,NaN,absent,absent,0,0,0,0,[],[],"['68034431', '90854551', '207085169', '2784724...",1.0
46,1217878555041976321,A4dWphyXVYrIL559EefcAKhCN3jSoarzHtaTscPevGs=,A4dWphyXVYrIL559EefcAKhCN3jSoarzHtaTscPevGs=,A4dWphyXVYrIL559EefcAKhCN3jSoarzHtaTscPevGs=,NaN,"Politikacılar gelecek seçimleri düşünür, devle...",NaN,369,435,2019-10-22,tr,tr,"Bunlar yaşanırken CHP ve İP kurumsal yapısına,...",2020-01-16 18:37,Twitter for

In [211]:
# remove unnecessary columns

merged_data = merged_data.drop([
    'latitude', # always 'absent' in positive dataset
    'longitude',
    'user_profile_url', # always 'NaN in positive dataset
    'in_reply_to_userid',
    'in_reply_to_tweetid',
    'quoted_tweet_tweetid',
    'retweet_userid',
    'retweet_tweetid',
    'quote_count', # can’t access these without a premium developer account
    'reply_count' 
], axis = 1)

In [212]:
merged_data.columns

Index(['tweetid', 'userid', 'user_display_name', 'user_screen_name',
       'user_reported_location', 'user_profile_description', 'follower_count',
       'following_count', 'account_creation_date', 'account_language',
       'tweet_language', 'tweet_text', 'tweet_time', 'tweet_client_name',
       'is_retweet', 'like_count', 'retweet_count', 'hashtags', 'urls',
       'user_mentions', 'label'],
      dtype='object')

In [213]:
# preprocessing tweet_text

import preprocessor as p
import emoji
import string

# remove url, smiley, hashtag, mention and reserved
p.set_options(p.OPT.URL, p.OPT.SMILEY, p.OPT.HASHTAG, p.OPT.MENTION, p.OPT.RESERVED)
for count,text in enumerate(merged_data['tweet_text']):
    merged_data.loc[count,'clean_text'] = p.clean(str(text))

# remove emojis  
def remove_emoji(text):
    return emoji.get_emoji_regexp().sub(u'', text)

for count,text in enumerate(merged_data['clean_text']):
    merged_data.loc[count,'clean_text'] = remove_emoji(text)

# remove extra white spaces, punctuation and apply lower casing
merged_data['clean_text'] = merged_data['clean_text'].str.lower().str.replace('[^\w\s]',' ').str.replace('\s\s+', ' ')


<ipython-input-213-0a1630ca9d48>:20: FutureWarning: The default value of regex will change from True to False in a future version.
  merged_data['clean_text'] = merged_data['clean_text'].str.lower().str.replace('[^\w\s]',' ').str.replace('\s\s+', ' ')


In [214]:
# preprocessing user_display_name, user_profile_description

# remove url, smiley, hashtag, mention and reserved
p.set_options(p.OPT.URL, p.OPT.SMILEY, p.OPT.HASHTAG, p.OPT.MENTION, p.OPT.RESERVED)

for count,text in enumerate(merged_data['user_display_name']):
    merged_data.loc[count,'user_display_name'] = p.clean(str(text))
    merged_data.loc[count,'user_display_name'] = remove_emoji(str(text))
    
for count,text in enumerate(merged_data['user_profile_description']):
    merged_data.loc[count,'user_profile_description'] = p.clean(str(text))
    merged_data.loc[count,'user_profile_description'] = remove_emoji(str(text))

merged_data['user_display_name'] = merged_data['user_display_name'].str.lower().str.replace('[^\w\s]',' ').str.replace('\s\s+', ' ')
merged_data['user_profile_description'] = merged_data['user_profile_description'].str.lower().str.replace('[^\w\s]',' ').str.replace('\s\s+', ' ')


<ipython-input-214-11d9912f9b4c>:14: FutureWarning: The default value of regex will change from True to False in a future version.
  merged_data['user_display_name'] = merged_data['user_display_name'].str.lower().str.replace('[^\w\s]',' ').str.replace('\s\s+', ' ')
<ipython-input-214-11d9912f9b4c>:15: FutureWarning: The default value of regex will change from True to False in a future version.
  merged_data['user_profile_description'] = merged_data['user_profile_description'].str.lower().str.replace('[^\w\s]',' ').str.replace('\s\s+', ' ')


In [215]:
# remove [,],',NaN from hashtags, urls, user_mentions

merged_data['hashtags'] = merged_data['hashtags'].str.replace('[','').str.replace(']','').str.replace("'",'').str.lower()
merged_data['urls'] = merged_data['urls'].str.replace('[','').str.replace(']','').str.replace("'",'')
merged_data['user_mentions'] = merged_data['user_mentions'].str.replace('[','').str.replace(']','').str.replace("'",'')

merged_data = merged_data.fillna('')


<ipython-input-215-3ec5a5efaf23>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  merged_data['hashtags'] = merged_data['hashtags'].str.replace('[','').str.replace(']','').str.replace("'",'').str.lower()
<ipython-input-215-3ec5a5efaf23>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  merged_data['urls'] = merged_data['urls'].str.replace('[','').str.replace(']','').str.replace("'",'')
<ipython-input-215-3ec5a5efaf23>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  merged_data['user_mentions'] = merged_data['user_mentions']

In [216]:
merged_data

,tweetid,userid,user_display_name,user_screen_name,user_reported_location,user_profile_description,follower_count,following_count,account_creation_date,account_language,tweet_language,tweet_text,tweet_time,tweet_client_name,is_retweet,like_count,retweet_count,hashtags,urls,user_mentions,label,clean_text
0,1221959349108604931,GBhlw1djPJ+Je62XTvtfBvAyqr1fggxCfgPTRlnmtmI=,gbhlw1djpj je62xtvtfbvayqr1fggxcfgptrlnmtmi,GBhlw1djPJ+Je62XTvtfBvAyqr1fggxCfgPTRlnmtmI=,Hell,bu profilde ürün yerleştirme bulunmaktadır kiş...,277,198,2012-09-09,tr,tr,Kendimi kandıracak bir yalan ararken tüm masum...,2020-01-28 00:53,Twitter for Android,False,29,0,,,,1.0,kendimi kandıracak bir yalan ararken tüm masum...
1,1214978416564420608,yKKIQh8NOLSz7artPJDPJB4qTzcEIqyqrxTEKsKEnM=,ykkiqh8nolsz7artpjdpjb4qtzceiqyqrxtekskenm,yKKIQh8NOLSz7artPJDPJB4qTzcEIqyqrxTEKsKEnM=,,çeke hapis kalksın,261,365,2019-12-26,en,tr,"ESNAFIN ONURU, GURURU İLE OYNAMAYIN\nEkonomini...",2020-01-08 18:33,Twitter for iPhone,False,0,0,,,"68034431, 564749194, 744924164, 243705279",1.0,esnafin onuru gururu i le oynamayin ekonominin...
2,1216791991171854336,yKKIQh8NOLSz7artPJDPJB4qTzcEIqyqrxTEKsKEnM=,ykkiqh8nolsz7artpjdpjb4qtzceiqyqrxtekskenm,yKKIQh8NOLSz7artPJDPJB4qTzcEIqyqrxTEKsKEnM=,,çeke hapis kalksın,261,365,2019-12-26,en,tr,"ESNAFIN ÖZGÜRLÜĞÜNÜ VERİN, \nSİCİLİNİ TEMİZLEY...",2020-01-13 18:39,Twitter for iPhone,False,0,2,,,"68034431, 90854551, 207085169, 278472416",1.0,esnafin özgürlüğünü veri n si ci li ni temi zl...
3,1217511840835690501,yKKIQh8NOLSz7artPJDPJB4qTzcEIqyqrxTEKsKEnM=,ykkiqh8nolsz7artpjdpjb4qtzceiqyqrxtekskenm,yKKIQh8NOLSz7artPJDPJB4qTzcEIqyqrxTEKsKEnM=,,çeke hapis kalksın,261,365,2019-12-26,en,tr,MAĞDUR ESNAF 5 yıl hapis cezasını hak etmiyor....,2020-01-15 18:20,Twitter for iPhone,False,0,2,,,"68034431, 895250727039369217, 1421299758, 3435...",1.0,mağdur esnaf 5 yıl hapis cezasını hak etmiyor ...
4,1213192530998759424,yKKIQh8NOLSz7artPJDPJB4qTzcEIqyqrxTEKsKEnM=,ykkiqh8nolsz7artpjdpjb4qtzceiqyqrxtekskenm,yKKIQh8NOLSz7artPJDPJB4qTzcEIqyqrxTEKsKEnM=,,çeke hapis kalksın,261,365,2019-12-26,en,tr,REKOR KIRILDI\nÖdenemeyen ÇEKLER %273 ARTTI\nH...,2020-01-03 20:16,Twitter for iPhone,False,0,0,,,"145941531, 887742723444420617, 3025338878, 101...",1.0,rekor kirildi ödenemeyen çekler 273 artti hazi...
5,1219329215499329538,yKKIQh8NOLSz7artPJDPJB4qTzcEIqyqrxTEKsKEnM=,ykkiqh8nolsz7artpjdpjb4qtzceiqyqrxtekskenm,yKKIQh8NOLSz7artPJDPJB4qTzcEIqyqrxTEKsKEnM=,,çeke hapis kalksın,261,365,2019-12-26,en,tr,Ekonominin büyümesinde en büyük katkı da esnaf...,2020-01-20 18:41,Twitter for iPhone,False,0,1,,,"68034431, 1421299758, 967087600426209281, 1461...",1.0,ekonominin büyümesinde en büyük katkı da esnaf...
6,1218235600605794305,yKKIQh8NOLSz7artPJDPJB4qTzcEIqyqrxTEKsKEnM=,ykkiqh8nolsz7artpjdpjb4qtzceiqyqrxtekskenm,yKKIQh8NOLSz7artPJDPJB4qTzcEIqyqrxTEKsKEnM=,,çeke hapis kalksın,261,365,2019-12-26,en,tr,"Fetö Kalkışması,\nPapaz Krizi,\nDöviz Artışı,\...",2020-01-17 18:16,Twitter for iPhone,False,0,0,,,"68034431, 3160777371, 34350322, 254777933",1.0,fetö kalkışması papaz krizi döviz artışı seçim...
7,1213190723367030789,yKKIQh8NOLSz7artPJDPJB4qTzcEIqyqrxTEKsKEnM=,ykkiqh8nolsz7artpjdpjb4qtzceiqyqrxtekskenm,yKKIQh8NOLSz7artPJDPJB4qTzcEIqyqrxTEKsKEnM=,,çeke hapis kalksın,261,365,2019-12-26,en,tr,2016 da yasalaşan 5941 ÇEK YASASI yüzünden bin...,2020-01-03 20:09,Twitter for iPhone,False,0,2,,,"145941531, 887742723444420617, 3025338878, 101...",1.0,2016 da yasalaşan 5941 çek yasasi yüzünden bin...
8,1218234581536645121,yKKIQh8NOLSz7artPJDPJB4qTzcEIqyqrxTEKsKEnM=,ykkiqh8nolsz7artpjdpjb4qtzceiqyqrxtekskenm,yKKIQh8NOLSz7artPJDPJB4qTzcEIqyqrxTEKsKEnM=,,çeke hapis kalksın,261,365,2019-12-26,en,tr,HAPİS CAYDIRICI OLSAYDI YÜZBİNLERCE ESNAFIN ÇE...,2020-01-17 18:12,Twitter for iPhone,False,7,38,,https://www.riskmerkezi.org/Content/Upload/ist...,"145941531, 887742723444420617, 3025338878, 101...",1.0,hapi s caydirici olsaydi yüzbi nlerce esnafin ...
9,1219331643003015169,yKKIQh8NOLSz7artPJDPJB4qTzcEIqyqrxT